# Advanced Design of a Hexapod!

The code below is an example project to design a hexapod assembly (group of parts). There's a lot going on down below, but you don't need to worry about the details! Simply run each cell and see what comes out. You can also click the `Run All` button above to run each cell back-to-back.

Since this is an assembly, you can also explore the `Explode` button in the viewer (there should be a play button that appears in the bottom right corner of the viewer)!

**Note:** You may be asked to install 3rd party software. If you get this request, please press `OK` to proceed.

In [ ]:
# Import necessary modules and functions
from ocp_vscode import *
import cadquery as cq
from jupyter_cadquery import (
    open_viewer, web_color,
    set_defaults, get_defaults, 
    PartGroup, Part, Faces, Edges
)
from cadquery_massembly import Mate, MAssembly, relocate

# Set the communication port for the viewer
set_port(3939)

# Open a viewer window with specified parameters
viewer = open_viewer("Hexapod", cad_width=800, height=600, theme="browser")


In [ ]:
# Set default visualization settings
set_defaults(
    axes=False,    # Hide the coordinate axes
    axes0=True,    # Display the global coordinate axes
    mate_scale=5,  # Scale factor for displaying mates
    transparent=False  # Disable transparency in visualization
)


# Hexapod 

In [ ]:
import numpy as np

# Define the horizontal angle
horizontal_angle = 25

# Function to generate intervals
def intervals(count):
    # Calculate intervals for equally dividing a circle
    r = [min(180, (90 + i * (360 // count)) % 360) for i in range(count)]
    return r

# Function to generate time intervals
def times(end, count):
    return np.linspace(0, end, count + 1)

# Function to generate vertical movement
def vertical(count, end, offset, reverse):
    # Calculate intervals for vertical movement
    ints = intervals(count)
    # Calculate heights for each interval
    heights = [round(35 * np.sin(np.deg2rad(x)) - 15, 1) for x in ints]
    # Append the first height to the end for closed loop
    heights.append(heights[0])
    # Adjust heights based on offset and reverse direction
    return times(end, count), heights[offset:] + heights[1:offset + 1]

# Function to generate horizontal movement
def horizontal(end, reverse):
    # Define factor for adjusting direction based on reverse flag
    factor = 1 if reverse else -1
    # Generate time intervals and angles for horizontal movement
    return times(end, 4), [0, factor * horizontal_angle, 0, -factor * horizontal_angle, 0]

# Print leg group 1 with transparent and filled settings
print("Leg group 1 (transparent)")
print("horizontal movement    ", horizontal(4, True))
print("vertical heights (left) ", vertical(8, 4, 0, True))
print("vertical heights (right)", vertical(8, 4, 0, False))

print("\nLeg group 1 (filled)")
print("horizontal movement", horizontal(4, False))
print("vertical heights (left) ", vertical(8, 4, 4, True))
print("vertical heights (right)", vertical(8, 4, 4, False))


# Assembly

## Parts

In [ ]:
# Define dimensions and parameters
thickness = 2   # Thickness of the object
height = 40     # Height of the object
width = 65      # Width of the object
length = 100    # Length of the object
diam = 4        # Diameter of a hole or circle
tol = 0.05      # Tolerance value for precision or clearance


In [ ]:
def create_base():
    # Define positions of base holes and stand holes
    x1, x2 = 0.63, 0.87
    base_holes = {
        "right_front": (-x1 * width, -x1 * length),
        "right_middle": (-x2 * width, 0),
        "right_back": (-x1 * width, x1 * length),
        "left_front": (x1 * width, -x1 * length),
        "left_middle": (x2 * width, 0),
        "left_back": (x1 * width, x1 * length),
    }
    stand_holes = {"front_stand": (0, -0.75 * length), "back_stand": (0, 0.8 * length)}

    # Create base geometry
    workplane = cq.Workplane()
    base = (workplane
            .ellipseArc(width, length, 295, 245, startAtCurrent=False).close()
            .pushPoints(list(base_holes.values())).circle(diam / 2 + tol)
            .moveTo(*stand_holes["back_stand"]).rect(width / 2 + 2 * tol, thickness + 2 * tol)
            .moveTo(*stand_holes["front_stand"]).rect(width / 2 + 2 * tol, thickness + 2 * tol)
            .extrude(thickness)
            )

    # Tagging faces and holes
    base.faces("<Z").tag("bottom")
    base.faces(">Z").tag("top")

    for name, hole in base_holes.items():
        base.faces("<Z").wires(cq.NearestToPointSelector(hole)).tag(name)

    for name, hole in stand_holes.items():
        base.faces("<Z").wires(cq.NearestToPointSelector(hole)).tag(name)

    return base

# Define names of base holes for reference
base_holes_names = {"right_back", "right_middle", "right_front", "left_back", "left_middle", "left_front"}


In [ ]:
def create_stand():
    # Create stand geometry components
    stand = cq.Workplane().box(height, width / 2 + 10, thickness)
    inset = cq.Workplane().box(thickness, width / 2, thickness)
    backing = cq.Workplane("ZX").polyline([(10, 0), (0, 0), (0, 10)]).close().extrude(thickness)

    # Assemble stand components
    stand = (stand
             .union(inset.translate(((height + thickness) / 2, 0, 0)))
             .union(inset.translate((-(height + thickness) / 2, 0, 0)))
             .union(backing.translate((-height / 2, -thickness / 2, thickness / 2)))
             .union(backing.rotate((0, 0, 0), (0, 1, 0), -90).translate((height / 2, -thickness / 2, thickness / 2)))
             )
    return stand

# Define names of stand components for reference
stand_names = ("front_stand", "back_stand")


In [ ]:
def create_upper_leg():
    # Define upper leg dimensions
    l1, l2 = 50, 80
    pts = [(0, 0), (0, height / 2), (l1, height / 2 - 5), (l2, 0)]
    upper_leg_hole = (l2 - 10, 0)

    # Create upper leg geometry
    upper_leg = (cq.Workplane()
                 .polyline(pts).mirrorX()
                 .pushPoints([upper_leg_hole]).circle(diam / 2 + tol).extrude(thickness)
                 .edges("|Z and (not <X)").fillet(4)
                 )

    # Create axle for the upper leg
    axle = (cq.Workplane("XZ", origin=(0, height / 2 + thickness + tol, thickness / 2))
            .circle(diam / 2).extrude(2 * (height / 2 + thickness + tol))
            )

    # Combine upper leg and axle
    upper_leg = upper_leg.union(axle)

    # Tag mating points
    upper_leg.faces(">Z").edges(cq.NearestToPointSelector(upper_leg_hole)).tag("top")
    upper_leg.faces("<Z").edges(cq.NearestToPointSelector(upper_leg_hole)).tag("bottom")

    return upper_leg


def create_lower_leg():
    # Define lower leg dimensions
    w, l1, l2 = 15, 20, 120
    pts = [(0, 0), (l1, w), (l2, 0)]
    lower_leg_hole = (l1 - 10, 0)

    # Create lower leg geometry
    lower_leg = (cq.Workplane()
                 .polyline(pts).mirrorX()
                 .pushPoints([lower_leg_hole]).circle(diam / 2 + tol)
                 .extrude(thickness)
                 .edges("|Z").fillet(5)
                 )

    # Tag mating points
    lower_leg.faces(">Z").edges(cq.NearestToPointSelector(lower_leg_hole)).tag("top")
    lower_leg.faces("<Z").edges(cq.NearestToPointSelector(lower_leg_hole)).tag("bottom")

    return lower_leg


# Define leg angles and names
leg_angles = {
    "right_back": -105, "right_middle": -90, "right_front": -75,
    "left_back": 105, "left_middle": 90, "left_front": 75,
}

leg_names = list(leg_angles.keys())


In [ ]:
# Create base, stand, upper leg, and lower leg components
base = create_base()
stand = create_stand()
upper_leg = create_upper_leg()
lower_leg = create_lower_leg()

# Display the components in the viewer
cv = show(
    base,
    stand.translate((100, -60, thickness / 2)),
    upper_leg.translate((80, 0, 0)),
    lower_leg.translate((75, 60, 0)),
)


## Define Assembly

In [ ]:
def create_hexapod():
    # Define shortcuts for readability
    L = lambda *args: cq.Location(cq.Vector(*args))
    C = lambda name: web_color(name)

    # Create leg assembly
    leg = (
        MAssembly(upper_leg, name="upper", color=C("orange"))
        .add(lower_leg, name="lower", color=C("orange"), loc=L(80, 0, 0))
    )

    # Create hexapod assembly
    hexapod = (
        MAssembly(base, name="bottom", color=C("silver"))
        .add(base, name="top", color=C("gainsboro"), loc=L(0, -2 * length, 0))
        .add(stand, name="front_stand", color=C("SkyBlue"), loc=L(120, -255, 0))
        .add(stand, name="back_stand", color=C("SkyBlue"), loc=L(180, -255, 0))
    )

    # Add legs to the hexapod assembly
    for i, name in enumerate(leg_names):
        hexapod.add(leg, name=name, loc=L(100, -55 * (i - 1.4), 0))

    return hexapod


## Define Mates 

In [ ]:
from collections import OrderedDict as odict

# Create the hexapod assembly
hexapod = create_hexapod()

# Add mates to the hexapod assembly
hexapod.mate("bottom?top", name="bottom", origin=True)
hexapod.mate("top?bottom", name="top", origin=True, transforms=odict(rx=180, tz=-(height + 2 * tol)))

# Add mates for stand assemblies
for name in stand_names:
    hexapod.mate(f"bottom?{name}", name=f"{name}_bottom", transforms=odict(rz=180 if "f" in name else 0))
    hexapod.mate(f"{name}@faces@<X", name=name, origin=True, transforms=odict(rx=180))

# Add mates for base holes
for name in base_holes_names:
    hexapod.mate(f"bottom?{name}", name=f"{name}_hole", transforms=odict(rz=leg_angles[name]))

# Add mates for leg assemblies
for name in leg_names:
    lower, upper, angle = ("top", "bottom", -75) if "left" in name else ("bottom", "top", -75)
    hexapod.mate(f"{name}?{upper}", name=f"leg_{name}_hole", transforms=odict(rz=angle))
    hexapod.mate(f"{name}@faces@<Y", name=f"leg_{name}_hinge", origin=True, transforms=odict(rx=180))
    hexapod.mate(f"{name}/lower?{lower}", name=f"leg_{name}_lower_hole", origin=True)

# Show the hexapod assembly with mates rendered
cv = show(hexapod, render_mates=True)


## Relocate and assemble

In [ ]:
# Relocate the hexapod assembly
hexapod.relocate()

# Show the relocated hexapod assembly
cv = show(hexapod)


In [ ]:
# Assemble the lower leg holes and hinges for each leg
for leg in leg_names:
    hexapod.assemble(f"leg_{leg}_lower_hole", f"leg_{leg}_hole")
    hexapod.assemble(f"leg_{leg}_hinge", f"{leg}_hole")

# Assemble the top and bottom parts of the hexapod
hexapod.assemble("top", "bottom")

# Assemble the stands
for stand_name in stand_names:
    hexapod.assemble(f"{stand_name}", f"{stand_name}_bottom")

# Show the hexapod assembly with specific rendering settings
cv = show(hexapod, render_mates=False, grid=[True, False, False], axes=False, render_normals=False)
